In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv('./data_down/1223_fifo_2wk.csv', sep = '\001')
df.columns = pd.Series(df.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
df2 =  pd.read_csv('./data_down/fifo_out_2w.csv', sep = '\001')
df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')
# df = df.iloc[:,6:]
# df = df2

df3 = pd.read_csv('./data_up/fifofefo_20220103_m.csv', sep = '|')
df3.columns = pd.Series(df3.columns).str.replace('.+\.', '')

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_4168/3798238643.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = pd.Series(df.columns).str.replace('.+\.', '')
C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_4168/3798238643.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df2.columns = pd.Series(df2.columns).str.replace('.+\.', '')
C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_4168/3798238643.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df3.columns = pd.Series(df3.columns).str.replace('.+\.', '')


In [4]:
ts = 'HPPXXWHWDS'

In [25]:
# df3 = df3.iloc[22:, :]
df3.head(2)
# df3['new_export_date'].str.match('(\d{4}\-\d{2}\-\d{2})')

# df3\
    # .query("ou_code != 'MICHETCTGS' & ou_code != 'MICHESHXCS'").to_csv('./data_up/fifofefo_20220103_m2.csv')

,ou_code,wms_warehouse_id,fifo_fefo,start_of_week,end_of_week,sku,old_remain_amt,old_remain_location,old_remain_date,old_remain_lock_code,new_export_amt,new_export_location,new_export_date,new_export_lock_codes,inc_day
0,COSTASHHTS,COSTA_SH,fefo,20211227,20220103,COSTA020655,1831.0,D-115-05-03-A,2022-03-20,Available,4145.0,D-115-05-03-A,2022-03-21,Available,20220103
1,COSTASHHTS,COSTA_SH,fefo,20211227,20220103,COSTA020745,286.0,"D-115-05-03-A,SHIP-01",2022-03-14,Available,1972.0,D-115-05-03-A,2022-03-21,Available,20220103


In [45]:
def data_(stats):
    """
    对数据筛选分类
    """
    fefo = df[df['fifo_fefo'] == stats].sort_values(['sku' , 'received_date' ], ascending=True)
    fefo = fefo[fefo['week_7']!=0]
    fefo['out_amt'] = fefo['week_6'] - fefo['week_7']


    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week']
        )['sku'].transform('count')
    fefo = fefo[fefo['flag'] != 1]

    fefo['flag'] = fefo.groupby(
        ['sku', 'ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week']
        )['out_amt'].transform(min)
    fefo = fefo[fefo['flag'] > 0]
    fefo.tail(8)
    return fefo
# def data2_(stats, df):
#     fefo = df
#     if stats == 'fifo':
#         fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
#     else:
#         fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
#     fefo_out = fefo.groupby(
#         ['ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week', 'sku'] , as_index= False
#         ).agg(
#             old_remain_amt = ('week_7', 'first'), # old remain
#             old_remain_location = ('location', 'first'), # 易过期
#             old_remain_date = ('received_date', 'first'), 
#             old_remain_lock_code = ('lock_codes', 'first'), 

#             new_export_amt = ('out_amt', 'mean'),  # new export
#             new_export_location = ('location', 'last'),
#             new_export_date = ('received_date', 'last'), 
#             new_export_lock_codes = ('lock_codes', 'last'), 
#     )
#     return fefo_out

def data2_(stats, df):
        fefo = df
        if stats == 'fifo':
            fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True)
        else:
            fefo = fefo.sort_values(['sku' , 'received_date'], ascending=True) 
            # ? 搞不懂发生了什么. 但这样的结果才是正确的..
        fefo_out = fefo.groupby(
            ['ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week', 'end_of_week', 'sku'],
            as_index= False
        #     ).agg(
        #         old_remain_amt = ('week_7', 'first'), # old remain
        #         old_remain_location = ('location', 'first'), # 易过期
        #         old_remain_date = ('received_date', 'first'), 
        #         old_remain_lock_code = ('lock_codes', 'first'), 

        #         new_export_amt = ('out_amt', 'mean'),  # new export
        #         new_export_location = ('location', 'last'),
        #         new_export_date = ('received_date', 'last'), 
        #         new_export_lock_codes = ('lock_codes', 'first'), 
        # )   # version failed.
            ).agg({
                'week_7' : [('old_remain_amt', 'first')], 
                'out_amt' : [('new_export_amt', 'first')], 
                'location': [('old_remain_location', 'first'), ('new_export_location', 'last')], 
                'received_date': [('old_remain_date','first'), ('new_export_date', 'last')],
                'lock_codes':[('old_remain_lock_code','first'), ('new_export_lock_codes', 'last')],
                }
        )
        return fefo_out

In [121]:
# fefo_out = fefo_out[~fefo_out['old_remain_location'].str.contains(fefo_out['new_export_location'][0])]

In [46]:
fifo = data_('fifo')
fefo = data_('fefo')
# fifo.head(5).sort_values(['sku' , 'received_date'], ascending=False)

In [47]:
df_out = pd.concat([
    data2_('fefo', fefo), data2_('fifo', fifo)
    ], axis = 0)\
        # .to_csv('./data_up/fifo_fefo_alert_thur.csv', index = None, encoding = 'utf_8_sig')

In [48]:
df_out.columns = list(df_out.columns.get_level_values(0)[0:6]) + list(df_out.columns.get_level_values(1)[6:])

In [16]:
from datetime import date 
def allsundays(year):
        """
        十年, 
        """
        return pd.Series(pd.date_range(start=str(year), end=str(year+10), 
                            freq="W-FRI").strftime('%Y%m%d'))
fridays = tuple([
        i for i in list(
            allsundays(2021)[allsundays(2021) < date.today().strftime('%Y%m%d')][-int(1):])])[0]

In [19]:
df_out.columns

Index(['ou_code', 'wms_warehouse_id', 'fifo_fefo', 'start_of_week',
       'end_of_week', 'sku', 'old_remain_amt', 'old_remain_location',
       'old_remain_date', 'old_remain_lock_code', 'new_export_amt',
       'new_export_location', 'new_export_date', 'new_export_lock_codes'],
      dtype='object')